In [35]:
import sys
sys.path.append("..")

In [1]:
import sklearn
import sklearn.preprocessing

In [2]:
import pandas as pd

snitz_ravia = pd.read_csv("Data/Challenge/Snitz_Ravia_Definition.csv").dropna()
snitz_ravia["Mixture Label"] = snitz_ravia["Mixture Label"].astype(int)
snitz_ravia

,Dataset,Mixture Label,CID,CID.1,CID.2,CID.3,CID.4,CID.5,CID.6,CID.7,...,CID.48,CID.49,CID.50,CID.51,CID.52,CID.53,CID.54,CID.55,CID.56,CID.57
1,Snitz 1,1,6501.0,264.0,2879.0,7685.0,7731.0,326.0,7888.0,61138.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Snitz 1,2,240.0,93009.0,323.0,8148.0,7762.0,3314.0,460.0,6184.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Snitz 1,3,7710.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Snitz 1,4,31276.0,93009.0,11002.0,323.0,7966.0,8148.0,7632.0,22201.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Snitz 1,5,10890.0,93009.0,11002.0,6982.0,323.0,8797.0,7966.0,8148.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Ravia,116,7799.0,31244.0,1031.0,7363.0,14286.0,8025.0,19310.0,7793.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
359,Ravia,117,6590.0,1031.0,176.0,180.0,7363.0,7150.0,14286.0,8025.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361,Ravia,118,1068.0,1031.0,180.0,3776.0,14286.0,61138.0,8025.0,6561.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363,Ravia,119,7799.0,3776.0,264.0,62336.0,6569.0,10448.0,261.0,5284503.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
def extract_mixture_labels_and_cids(df):
    result = dict()
    cid_columns = [col for col in df.columns if col.startswith('CID')]

    for index, row in df.iterrows():
        mixture_label = (row["Dataset"]+"_" if "Dataset" in row else "") + str(row['Mixture Label'])
        cids = [int(cid) for cid in row[cid_columns].tolist() if cid != 0]
        assert len(set(cids)) == len(cids)
        
        
        result[mixture_label] = cids

    return result

def extract_mixture_labels(df):
    result = []
    cid_columns = [col for col in df.columns if col.startswith('CID')]

    for index, row in df.iterrows():
        mixture_label = (row["Dataset"]+"_" if "Dataset" in row else "") + str(row['Mixture Label'])
        result.append(mixture_label)

    return result
        
snitz_ravia_mixtures = extract_mixture_labels_and_cids(snitz_ravia)
len(snitz_ravia_mixtures), next(iter(snitz_ravia_mixtures.items()))

(183,
 ('Snitz 1_1', [6501, 264, 2879, 7685, 7731, 326, 7888, 61138, 8030, 1183]))

In [4]:
import json

report = pd.read_csv("Data/report.csv")
assert ~report["Multiple_matches"].all()
report["Label"] = report.apply(lambda x: f"Bushdid_{x['Idx_processed_dream_bushdid']+1}", axis=1)
bushdid_mixtures = report.set_index('Label')['bushdid_CIDs'].to_dict()
bushdid_mixtures = {k:json.loads(cids) for k, cids in bushdid_mixtures.items()}
len(bushdid_mixtures), next(iter(bushdid_mixtures.items()))

(520,
 ('Bushdid_1',
  [440917,
   5281515,
   8030,
   3314,
   31272,
   798,
   6184,
   8892,
   8025,
   6054,
   7463,
   8148,
   999,
   1060,
   22873,
   10890,
   61138,
   2879,
   10430,
   8048,
   6561,
   22386,
   11509,
   443162,
   8797,
   16666,
   7749,
   7793,
   7799,
   7762]))

In [5]:
all_mixtures = snitz_ravia_mixtures | bushdid_mixtures
len(all_mixtures)

703

In [8]:
test_df = pd.read_csv("Data/Challenge/Mixure_Definitions_test_set.csv")
test_mixtures = extract_mixture_labels_and_cids(test_df)
len(test_mixtures), next(iter(test_mixtures.items()))

(92, ('1', [10890, 62465, 12180, 7799, 6054, 12367, 61016, 7410, 7749, 10722]))

In [22]:
train_experiments = pd.read_csv("Data/Challenge/TrainingData_mixturedist.csv").dropna()
train_experiments = train_experiments.rename(columns={"Mixture 1":"Mixture_1","Mixture 2":"Mixture_2","Experimental Values":"Experimental_Values"})
train_experiments

,Dataset,Mixture_1,Mixture_2,Experimental_Values
0,Snitz 1,1.0,2.0,0.604167
1,Snitz 1,1.0,3.0,0.651042
2,Snitz 1,1.0,5.0,0.505208
3,Snitz 1,1.0,6.0,0.411458
4,Snitz 1,1.0,7.0,0.562500
...,...,...,...,...
495,Bushdid,509.0,510.0,0.461538
496,Bushdid,513.0,514.0,0.692308
497,Bushdid,515.0,516.0,0.730769
498,Bushdid,517.0,518.0,0.538462


In [23]:
def make_df_with_cids(experiments,mixtures,use_dataset=True):
    experiments = experiments.copy()
    if use_dataset:
        experiments["Label 1"] = experiments.apply(lambda x: x["Dataset"]+"_"+str(int(x["Mixture_1"])),axis=1)
        experiments["Label 2"] = experiments.apply(lambda x: x["Dataset"]+"_"+str(int(x["Mixture_2"])),axis=1)
    else:
        experiments["Label 1"] = experiments.apply(lambda x: str(int(x["Mixture_1"])),axis=1)
        experiments["Label 2"] = experiments.apply(lambda x: str(int(x["Mixture_2"])),axis=1)
    experiments["CIDS 1"] = experiments.apply(lambda x: mixtures[x["Label 1"]],axis=1)
    experiments["CIDS 2"] = experiments.apply(lambda x: mixtures[x["Label 2"]],axis=1)
    return experiments
train_experiments_with_cids = make_df_with_cids(train_experiments,all_mixtures)
train_experiments_with_cids

,Dataset,Mixture_1,Mixture_2,Experimental_Values,Label 1,Label 2,CIDS 1,CIDS 2
0,Snitz 1,1.0,2.0,0.604167,Snitz 1_1,Snitz 1_2,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...","[240, 93009, 323, 8148, 7762, 3314, 460, 6184,..."
1,Snitz 1,1.0,3.0,0.651042,Snitz 1_1,Snitz 1_3,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...",[7710]
2,Snitz 1,1.0,5.0,0.505208,Snitz 1_1,Snitz 1_5,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...","[10890, 93009, 11002, 6982, 323, 8797, 7966, 8..."
3,Snitz 1,1.0,6.0,0.411458,Snitz 1_1,Snitz 1_6,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...","[7710, 31276, 10890, 240, 93009, 11002, 6982, ..."
4,Snitz 1,1.0,7.0,0.562500,Snitz 1_1,Snitz 1_7,"[6501, 264, 2879, 7685, 7731, 326, 7888, 61138...","[93009, 460, 443158, 6544]"
...,...,...,...,...,...,...,...,...
495,Bushdid,509.0,510.0,0.461538,Bushdid_509,Bushdid_510,"[176, 7059, 7654, 4133, 31272, 2969, 7344, 605...","[176, 7059, 7654, 4133, 31272, 2969, 7344, 605..."
496,Bushdid,513.0,514.0,0.692308,Bushdid_513,Bushdid_514,"[612, 6654, 19310, 22386, 62465, 11552, 7793, ...","[10430, 454, 440917, 3776, 240, 2758, 10821, 7..."
497,Bushdid,515.0,516.0,0.730769,Bushdid_515,Bushdid_516,"[16666, 19310, 62465, 7848, 440917, 7991, 1060...","[16666, 19310, 62465, 22201, 7921, 22386, 7583..."
498,Bushdid,517.0,518.0,0.538462,Bushdid_517,Bushdid_518,"[1127, 10821, 8858, 8148, 3776, 8048, 7731, 31...","[6561, 7921, 7654, 24915, 261, 7888, 999, 8030..."


In [59]:
train_experiments_with_cids['Dataset'].value_counts()

Dataset
Bushdid    234
Snitz 1    140
Snitz 2     81
Ravia       45
Name: count, dtype: int64

In [19]:
test_experiments = pd.read_csv("Data/Challenge/Test_set_Submission_form.csv").dropna()
test_experiments_with_cids = make_df_with_cids(test_experiments,test_mixtures,use_dataset=False)
test_experiments_with_cids.head(5)

,Dataset,Mixture_1,Mixture_2,Predicted_Experimental_Values,Label 1,Label 2,CIDS 1,CIDS 2
0,CGm01,1,17,0,1,17,"[10890, 62465, 12180, 7799, 6054, 12367, 61016...","[22201, 6184, 12741, 8797, 11509, 7793, 7770, ..."
1,CGm02,2,18,0,2,18,"[31244, 14286, 7714, 8892, 18827, 12020, 7519,...","[7410, 31276, 62433, 10821, 7799, 22201, 6590,..."
2,CGm03,3,19,0,3,19,"[7793, 8091, 8797, 26331, 22873, 31265, 8118, ...","[62433, 62336, 6590, 31244, 62465, 11002, 5634..."
3,CGm04,4,20,0,4,20,"[31272, 6501, 8091, 454, 31252, 6184, 7410, 86...","[8051, 7799, 3314, 8858, 12178, 4133, 8118, 10..."
4,CGm05,5,21,0,5,21,"[8892, 14286, 7991, 7749, 8051, 62465, 62336, ...","[7344, 7654, 10890, 6590, 10882, 11509, 7731, ..."


In [20]:
leaderboard_experiments = pd.read_csv("Data/Challenge/Leaderboard_set_Submission_form.csv").dropna()
leaderboard_experiments_with_cids = make_df_with_cids(leaderboard_experiments,all_mixtures)
leaderboard_experiments_with_cids.head(5)

,Dataset,Mixture_1,Mixture_2,Predicted_Experimental_Values,Label 1,Label 2,CIDS 1,CIDS 2
0,Bushdid,65,66,0,Bushdid_65,Bushdid_66,"[62336, 460, 8103, 6054, 1127, 637511, 7632, 1...","[62336, 460, 8103, 6054, 1127, 637511, 7632, 1..."
1,Snitz 2,1,8,0,Snitz 2_1,Snitz 2_8,"[326, 26331, 6544, 1140]","[31276, 8148, 7762, 18827, 7714]"
2,Snitz 1,24,17,0,Snitz 1_24,Snitz 1_17,"[7749, 61138, 6054, 6544]","[14286, 31276, 7600, 7519, 11002, 6982, 307, 3..."
3,Snitz 2,12,14,0,Snitz 2_12,Snitz 2_14,"[7410, 326, 2758, 62444, 7770, 1140]","[8797, 7731, 7966, 3314, 62336, 7059, 7991, 61..."
4,Snitz 1,49,45,0,Snitz 1_49,Snitz 1_45,"[7600, 62433, 7991, 6989]",[7685]


In [30]:
all_cids = set()
for _, cids in all_mixtures.items():
    all_cids.update(cids)
    
for _, cids in test_mixtures.items():
    all_cids.update(cids)
len(all_cids)

163

In [17]:
import json
with open("Data/cid_to_smiles.json") as f:
    cid_to_smiles = json.load(f)
cid_to_smiles = {int(cid):smiles for cid, smiles in cid_to_smiles.items()}
len(cid_to_smiles), next(iter(cid_to_smiles.items()))

(163, (176, 'CC(=O)O'))

In [18]:
for cid in all_cids:
    assert cid in cid_to_smiles

In [25]:
from ogb.utils import smiles2graph
import numpy as np
from torch_geometric.data import InMemoryDataset, download_url, Data
import torch
from tqdm.notebook import tqdm

def to_torch(graph):
    tensor_keys = ["edge_index", 'edge_feat', 'node_feat']
    for key in tensor_keys:
        graph[key] = torch.tensor(graph[key])
    return Data(x=graph["node_feat"].float(),edge_attr=graph["edge_feat"],edge_index=graph["edge_index"])

graph_data = dict()
for smiles in tqdm(cid_to_smiles.values(),smoothing=0):
    graph_data[smiles] = to_torch(smiles2graph(smiles))
len(graph_data), next(iter(graph_data.items()))

  0%|          | 0/163 [00:00<?, ?it/s]

(159, ('CC(=O)O', Data(x=[4, 9], edge_index=[2, 6], edge_attr=[6, 3])))

In [55]:
import data

def make_mixture(cids):
    return data.combine_graphs([graph_data[cid_to_smiles[cid]] for cid in cids])

def make_mixture_data(df_with_cids,use_y):
    mixture_data = []
    for _,row in tqdm(df_with_cids.iterrows(),total=len(df_with_cids)):
        graph1 = make_mixture(row["CIDS 1"])
        graph2 = make_mixture(row["CIDS 2"])
        
        pair_data = {"dataset":row['Dataset'],"label":f"{row['Label 1']}+{row['Label 2']}","graph1":graph1,"graph2":graph2}
        if use_y:
            pair_data["y"] = torch.tensor(row["Experimental_Values"])
        mixture_data.append(pair_data)
    return mixture_data

train_mixture_data = make_mixture_data(train_experiments_with_cids,use_y=True)
len(train_mixture_data), train_mixture_data[0]

  0%|          | 0/500 [00:00<?, ?it/s]

(500,
 {'dataset': 'Snitz 1',
  'label': 'Snitz 1_1+Snitz 1_2',
  'graph1': BlendData(x=[97, 9], edge_index=[2, 190], edge_attr=[190, 3], mol_batch=[97], blend_batch=[10]),
  'graph2': BlendData(x=[96, 9], edge_index=[2, 192], edge_attr=[192, 3], mol_batch=[96], blend_batch=[10]),
  'y': tensor(0.6042)})

In [56]:
test_mixture_data = make_mixture_data(test_experiments_with_cids,use_y=False)
len(test_mixture_data), test_mixture_data[0]

  0%|          | 0/46 [00:00<?, ?it/s]

(46,
 {'dataset': 'CGm01  ',
  'label': '1+17',
  'graph1': BlendData(x=[95, 9], edge_index=[2, 178], edge_attr=[178, 3], mol_batch=[95], blend_batch=[10]),
  'graph2': BlendData(x=[95, 9], edge_index=[2, 178], edge_attr=[178, 3], mol_batch=[95], blend_batch=[10])})

In [57]:
leaderboard_mixture_data = make_mixture_data(leaderboard_experiments_with_cids,use_y=False)
len(leaderboard_mixture_data), leaderboard_mixture_data[0]

  0%|          | 0/46 [00:00<?, ?it/s]

(46,
 {'dataset': 'Bushdid',
  'label': 'Bushdid_65+Bushdid_66',
  'graph1': BlendData(x=[285, 9], edge_index=[2, 546], edge_attr=[546, 3], mol_batch=[285], blend_batch=[30]),
  'graph2': BlendData(x=[254, 9], edge_index=[2, 478], edge_attr=[478, 3], mol_batch=[254], blend_batch=[30])})

In [60]:
import h5py
def make_file(fname,mixture_data):
    with h5py.File(fname, 'w') as f:
        for data in tqdm(mixture_data):
            group = f.create_group(data["label"])
            graph_group1 = group.create_group("graph1")
            for k,v in data["graph1"].items():
                graph_group1.create_dataset(k,data=v.numpy())
            graph_group2 = group.create_group("graph2")
            for k,v in data["graph2"].items():
                graph_group2.create_dataset(k,data=v.numpy())
            group.create_dataset("dataset",data=data["dataset"])
            if "y" in data:
                group.create_dataset("y",data=data["y"].numpy())
make_file("Data/train.h5",train_mixture_data)
make_file("Data/test.h5",test_mixture_data)
make_file("Data/leaderboard.h5",leaderboard_mixture_data)

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]